In [1]:
import tensorflow as tf
import numpy as np
import os
import importlib

In [2]:
from r2_bot import api, common_utils, model as mod_model, serving_io, train_io, training_utils

In [3]:
ds_dir = 'wrk'
train_fn = 'restroom-dataset.v2.train.tfrecords'
valid_fn = 'restroom-dataset.v2.valid.tfrecords'

In [4]:
model_dir = 'wrk/rnn-v2'
export_dir_base = os.path.join(model_dir, 'export')

In [21]:
!rm -rf wrk/rnn-v2

In [6]:
os.listdir('pretrained')

['araneum_upos_skipgram_300_2_2018.txt',
 'araneum_upos_skipgram_300_2_2018.npy',
 'rebuild.vectors.npy',
 'araneum_upos_skipgram_300_2_2018.vec.gz',
 'rebuild.vocab.txt']

In [7]:
vocab_fp = 'pretrained/rebuild.vocab.txt'
emb_fp = 'pretrained/rebuild.vectors.npy'

In [8]:
common_utils.get_lines(vocab_fp)

196622

In [9]:
emb_pretrained_arr = np.load(emb_fp)

In [10]:
emb_pretrained_arr.shape

(196622, 300)

In [11]:
MODEL_PARAM_DEFAULTS = {
    'pretrained.dtype' : tf.float32,
    'pretrained.dim' : emb_pretrained_arr.shape[1],
    'encoder.hidden_size' : 512,
    'l2_lambda': 0,
    'batch_size': 32
}

In [12]:
del emb_pretrained_arr

In [13]:
model_params = dict(vocab_filepath=vocab_fp, 
                    pretrained_path=emb_fp,
                    **MODEL_PARAM_DEFAULTS)

In [14]:
train_ds_size = train_io.get_ds_size(ds_dir, train_fn)
train_ds_size

90

In [15]:
steps_in_train_epoch = int(np.ceil(train_ds_size / model_params['batch_size']))
steps_in_train_epoch

3

In [16]:
estimator_config = tf.estimator.RunConfig(save_checkpoints_steps=steps_in_train_epoch)

In [22]:
importlib.reload(mod_model)

<module 'r2_bot.model' from '/home/ec2-user/SageMaker/r2_bot/model.py'>

In [23]:
estimator = tf.estimator.Estimator(mod_model.model_fn, model_dir=model_dir, params=model_params, config=estimator_config)

INFO:tensorflow:Using config: {'_model_dir': 'wrk/rnn-v2', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 3, '_save_checkpoints_secs': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f2bf20b6320>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


# Train

In [86]:
# TODO handle <pad> correctly in vocab

In [18]:
def train_input_fn(repeat):
    return train_io.input_fn(ds_dir, train_fn, model_params, repeat=repeat, shuffle_size=train_ds_size)

def eval_input_fn():
    return train_io.input_fn(ds_dir, valid_fn, model_params)

In [19]:
def train_and_evaluate_old_way(epoch_num):
    es_handler = training_utils.EarlyStoppingHandler(model_dir, 'accuracy', 
                                                     no_increase_checks=3,
                                                     no_increase_threshold=0.01)
    for e in range(epoch_num):
        if not es_handler.should_continue():
            break
        estimator.train(input_fn=lambda: train_input_fn(None))
        eval_results = estimator.evaluate(input_fn=eval_input_fn)
        print('\nEvaluation results:\n\t%s\n' % eval_results)
        es_handler.add_eval_dict(eval_results)
    best_step = es_handler.get_best_step()
    print("Exporting checkpoint for the best step", best_step)
    best_ckp_path = os.path.join(model_dir, 'model.ckpt-%s' % best_step.step)
    return estimator.export_savedmodel(export_dir_base, serving_io.serving_fn,
                               checkpoint_path=best_ckp_path)
    

In [24]:
train_and_evaluate_old_way(20)

INFO:tensorflow:Read 0 previous accuracy values: []
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Summary name word_embeddings:0 is illegal; using word_embeddings_0 instead.
INFO:tensorflow:Summary name encoder_rnn/basic_lstm_cell/kernel:0 is illegal; using encoder_rnn/basic_lstm_cell/kernel_0 instead.
INFO:tensorflow:Summary name encoder_rnn/basic_lstm_cell/bias:0 is illegal; using encoder_rnn/basic_lstm_cell/bias_0 instead.
INFO:tensorflow:Summary name label_logits/weight_matrix:0 is illegal; using label_logits/weight_matrix_0 instead.
INFO:tensorflow:Summary name label_logits/bias:0 is illegal; using label_logits/bias_0 instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Loaded pretrained embeddings weights (196622, 300) float32 from pretrained/rebuild.vectors.npy
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 i

b'wrk/rnn-v2/export/1546896355'

## Run Old Way

## Run New Way

In [129]:
early_stopping_hook = tf.contrib.estimator.stop_if_no_increase_hook(estimator, 'accuracy', 3,
                                                         min_steps=steps_in_train_epoch * 3,
                                                         run_every_secs=None,
                                                        run_every_steps=steps_in_train_epoch)

In [132]:
train_spec = tf.estimator.TrainSpec(
    lambda: train_input_fn(-1), 
    hooks=[
        early_stopping_hook
    ])

In [133]:
def _compare_fn(best_eval_result, current_eval_result):
    metric='accuracy'
    return best_eval_result[metric] > current_eval_result[metric]

best_exporter = tf.estimator.BestExporter(serving_input_receiver_fn=serving_fn, 
                                          compare_fn=_compare_fn,
                                          exports_to_keep=3)
eval_spec = tf.estimator.EvalSpec(eval_input_fn, steps=None, start_delay_secs=2, throttle_secs=2,
                                  exporters=best_exporter)

In [ ]:
tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

# LEGACY

In [17]:
[p for p in estimator.predict(tf.estimator.inputs.numpy_input_fn({
    'tokens' : np.array([['Я', 'хочц', 'поссикать', 'gfdgf']]),
    'token_lengths' : np.array([3])
}, shuffle=False))]

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from wrk/baseline-v0/model.ckpt-1
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[{'intent_i': 2,
  'confidence': 0.5903566,
  'input_length': 3,
  'intent': b'GoToHell'}]

In [15]:
#train_ds = input_fn(ds_dir, train_fn, 4, {})
#train_iter = train_ds.make_one_shot_iterator()
#train_record, train_label = train_iter.get_next()
#train_estim_spec = model_fn(train_record, train_label, 
#                            mode=tf.estimator.ModeKeys.TRAIN,
#                            params=dict(vocab_filepath=vocab_fp, **MODEL_PARAM_DEFAULTS))

In [16]:
#with tf.Session() as sess:
#    sess.run(tf.tables_initializer())
#    sess.run(tf.global_variables_initializer())
#    print(sess.run(sess.graph.get_tensor_by_name('word_indices:0')))
#    print(sess.run(
#        [sess.graph.get_tensor_by_name('true_intent_i:0'), 
#         train_estim_spec.loss]
#    ))
#    print(sess.run(
#        [sess.graph.get_tensor_by_name('true_intent_i:0'), 
#         train_estim_spec.loss]
#    ))